I wrote [a few months back](http://tdhopper.com/blog/2016/Nov/15/data-scientists-need-more-automation/) about how data scientists need more automation. In particular, I @@@@

Ansible provides "human readable automation" for "app deployment" and "configuration management". Unlike tools like Chef, it doesn't require an agent to be running on remote machines. In short, it translates declarative YAML files into shell commands and runs them your machines over SSH.

### Installing Ansible with Homebrew 

First, you'll need to install Ansible. I recommend doing this with [Homebrew](https://brew.sh/).

In [3]:
brew install ansible

brew install ansible
We do not provide support for this pre-release version.
You may encounter build failures or other breakages.


: 1

### Quickstart

Soon, I'll show you how to put write an Ansible YAML file. However, Ansible also allows you specify tasks from the command line. 

Here's how we could use Ansible ping our local host:

In [44]:
ansible -i 'localhost,' -c local -m ping all

ansible -i 'localhost,' -c local -m ping all
localhost | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


: 1

ansible -i 'localhost,' -c local -m ping all -vvv
Using /Users/tdhopper/repos/automating_python/ansible.cfg as config file
<localhost> ESTABLISH LOCAL CONNECTION FOR USER: tdhopper
<localhost> EXEC /bin/sh -c '( umask 77 && mkdir -p "` echo $HOME/.ansible/tmp/ansible-tmp-1490022689.35-148051626033129 `" && echo ansible-tmp-1490022689.35-148051626033129="` echo $HOME/.ansible/tmp/ansible-tmp-1490022689.35-148051626033129 `" ) && sleep 0'
<localhost> PUT /var/folders/4l/b_gx3vx957g8lw0g__5nz9_h0000gn/T/tmp6QUTxd TO /Users/tdhopper/.ansible/tmp/ansible-tmp-1490022689.35-148051626033129/ping
<localhost> EXEC /bin/sh -c 'LANG=en_US.UTF-8 LC_ALL=en_US.UTF-8 LC_MESSAGES=en_US.UTF-8 /usr/bin/python /Users/tdhopper/.ansible/tmp/ansible-tmp-1490022689.35-148051626033129/ping; rm -rf "/Users/tdhopper/.ansible/tmp/ansible-tmp-1490022689.35-148051626033129/" > /dev/null 2>&1 && sleep 0'
localhost | SUCCESS => {
    "changed": false, 
    "invocation": {
        "module_args": {
            "data": 

: 1

This command calls ansible and tells it:
* To use `localhost` as it's inventory (`-i`). Inventory is Ansible speak for machine or machines you want to be able to run commands on. 
* To connect (`-c`) locally (`local`) instead of over SSH. 
* To run the [`ping` module](http://docs.ansible.com/ansible/ping_module.html) (`-m`) to test the connection.
* To run the command on `all` hosts in the inventory (in this case, our inventory is just the `localhost`).

[Michael Booth](http://www.mechanicalfish.net/start-learning-ansible-with-one-line-and-no-files/) has a [post](http://www.mechanicalfish.net/start-learning-ansible-with-one-line-and-no-files/) that goes into more detail about this command.

Behind the scenes, Ansible is turning this `-m ping` command into shell commands. (Try running with the `-vvv` flag to see what's happening behind the scenes.) It can also execute arbitrary commands; by default, it'll use the Bourne shell `sh`. 

In [51]:
ansible all -i 'localhost, ' -c local -a "/bin/echo hello" 

ansible all -i 'localhost, ' -c local -a "/bin/echo hello"
localhost | SUCCESS | rc=0 >>
hello



: 1

### Setting up an Ansible Inventory

Instead of specifying our inventory with the `-i` flag each time, we should specify an Ansible inventory file. This file is a text file specifying machines you have SSH access to; you can also group machines under bracketed headings. For example:

```
mail.example.com

[webservers]
foo.example.com
bar.example.com

[dbservers]
one.example.com
two.example.com
three.example.com
```

Ansible has to be able to connect to these machines over SSH, so you will likely need to have relevant entries in your [`.ssh/config` file](http://nerderati.com/2011/03/17/simplify-your-life-with-an-ssh-config-file/).

By default, the Ansible CLI will look for a system-wide Ansible inventory file in `/etc/ansible/hosts`. You can also specify an alternative path for an intentory file with the `-i` flag.

For this tutorial, I'd like to have an inventory file specific to the project directory without having to specify it each time we call Ansible. We can do this by creating a file called `./ansible.cfg` and set the name of our local inventory file:

In [19]:
cat ./ansible.cfg

cat ./ansible.cfg
[defaults]
inventory = ./hosts

: 1

You can check that Ansible is picking up your config file by running `ansible --version`.

In [24]:
ansible --version

ansible --version
ansible 2.1.0.0
  config file = /Users/tdhopper/repos/automating_python/ansible.cfg
  configured module search path = Default w/o overrides


: 1

For this example, I just have one host, a [Digital Ocean VPS](https://www.digitalocean.com/). To run the examples below, you should create a VPS instance on Digital Ocean, [Amazon](https://amazonlightsail.com), or elsewhere; you'll want to configure it for [passwordless authentication](https://www.digitalocean.com/community/tutorials/how-to-set-up-ssh-keys--2). I have an entry like this in my `~/.ssh/hosts` file: 

```
Host digitalocean
  HostName 45.55.395.23
  User root
  Port 22
  IdentityFile /Users/tdhopper/.ssh/id_rsa
  ForwardAgent yes
```
  
and my intentory file (`~/hosts`) is just

```
digitalocean
```

Now I can verify that Ansible can connect to my machine by running the ping command. 

In [39]:
ansible all -m ping

ansible all -m ping
digitalocean | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


: 1

We told Ansible to run this command on `all` specified hosts in the inventory. It found our inventory by loading the `ansible.cfg` which specified `./hosts` as the inventory file.

### Writing our first Playbook

While adhoc commands will often be useful, the real power of Ansible comes from creating repeatable sets of instructions called [Playbooks](http://docs.ansible.com/ansible/playbooks.html).

A playbook contains a list of "plays". Each play specifies a set of tasks to be run and which hosts to run them on. A "task" is a call to an Ansible module, like the "ping" module we've already seen. Ansible [comes packaged with about 1000 modules](http://docs.ansible.com/ansible/list_of_all_modules.html) for all sorts of use cases. You can also extend it with your own [modules](http://docs.ansible.com/ansible/dev_guide/developing_modules.html) and [roles](http://docs.ansible.com/ansible/playbooks_roles.html#roles).

Our first playbook will just execute the ping module on all our hosts. It's a playbook with a single play comprised of a single task.

In [66]:
cat ping.yml

cat ping.yml
---
- hosts: all
  tasks:
  - name: ping all hosts
    ping:

: 1

We can run our playbook with the `ansible-playbook` command.

In [70]:
ansible-playbook ping.yml

ansible-playbook ping.yml
 ____________ 
< PLAY [all] >
 ------------ 
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

 ______________ 
< TASK [setup] >
 -------------- 
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

ok: [digitalocean]
 _______________________ 
< TASK [ping all hosts] >
 ----------------------- 
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

ok: [digitalocean]
 ____________ 
< PLAY RECAP >
 ------------ 
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

digitalocean               : ok=2    changed=0    unreachable=0    failed=0   



: 1

You might wonder why there are cows on your screen. You can find out [here](https://michaelheap.com/cowsay-and-ansible/). However, the important thing is that our task was executed and returned successfully.

We can override the hosts list for the play with the `-i` flag to see what the output looks like when Ansible fails to run the play.

In [72]:
ansible-playbook -i "fakehost, " ping.yml

ansible-playbook -i "fakehost, " ping.yml
 ____________ 
< PLAY [all] >
 ------------ 
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

 ______________ 
< TASK [setup] >
 -------------- 
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

fatal: [fakehost]: UNREACHABLE! => {"changed": false, "msg": "Failed to connect to the host via ssh.", "unreachable": true}
	to retry, use: --limit @ping.retry
 ____________ 
< PLAY RECAP >
 ------------ 
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

fakehost                   : ok=0    changed=0    unreachable=1    failed=0   



: 1